# Introduction to ESG Bulk - Python

ESG stands for Environmental, Social and (Corporate) Governance data.

Refinitiv Data Platform (RDP) provides simple web based API access to a broad range of content, including ESG content and ESG content in bulk.

With growing popularity of socially conscious investing, Refinitiv offers one of the most comprehensive Environmental, Social and Governance (ESG) databases in the industry, covering over 80% of global market cap, across more than 450 different ESG metrics, with history going back to 2002. Customers looking to download our ESG content can do so through our bulk API service in Refinitiv Data Platform (RDP). RDP is a cloud based API that provides a single access point to all Refinitiv content.

ESG data is the first content made available in our bulk API service known as Client File Store (CFS). This capability allows our customers to download our entire history of ESG coverage. To more about how the ESG Bulk Service works in Refinitiv Data Platform, please visit:

https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs

Within RDP family of service, ESG Bulk is part of Client File Store (CFS) - based section of service, find out more at:

https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis

Let us now focus on the programmatic interaction with ESG Bulk RDP service.

In [1]:
import requests, json, time, getopt, sys
import pandas as pd

### Set Valid Credentials 

Valid RDP credentials are required to proceed:
* USERNAME
* PASSWORD
* CLIENTID

To read one's valid credentials from a file (that can be shared by many code examples), leave below code as is.

To provide credentials in place:
* replace the commented credentials with one's valid assigned credentials
* comment the read from file step readCredsFromFile

In [2]:
USERNAME = "VALIDUSER"
PASSWORD = "VALIDPASSWORD"
CLIENT_ID = "SELFGENERATEDCLIENTID"

def readCredsFromFile(filePathName):
### Read valid credentials from file
    global USERNAME, PASSWORD, CLIENT_ID
    credFile = open(filePathName,"r")    # one per line
                                                #--- RDP MACHINE ID---
                                                #--- LONG PASSWORD---
                                                #--- GENERATED CLIENT ID---

    USERNAME = credFile.readline().rstrip('\n')
    PASSWORD = credFile.readline().rstrip('\n')
    CLIENT_ID = credFile.readline().rstrip('\n')

    credFile.close()

readCredsFromFile("..\creds\credFileHuman.txt")

# Uncomment - to make sure that creds are either set in code or read in correctly
#print("USERNAME="+str(USERNAME))
#print("PASSWORD="+str(PASSWORD))
#print("CLIENT_ID="+str(CLIENT_ID))

### Set Application Constants

In [3]:
# Set Application Constants
RDP_AUTH_VERSION = "/v1"
RDP_ESG_BULK_VERSION = "/v1"
RDP_BASE_URL = "https://api.refinitiv.com"
RDP_ESG_BUCKET = "bulk-esg"
CATEGORY_URL = "/auth/oauth2"
ENDPOINT_URL = "/token"
CLIENT_SECRET = ""
TOKEN_FILE = "token.txt"
SCOPE = "trapi"
FILESET_ID = ''
PACKAGE_ID = ''

### Define Token Handling and Obtain a Valid Token

Having a valid token is a pre-requisite to requesting of any RDP content, and will be passed into the next steps.

In [4]:
TOKEN_ENDPOINT = RDP_BASE_URL + CATEGORY_URL + RDP_AUTH_VERSION + ENDPOINT_URL

def _requestNewToken(refreshToken):
    if refreshToken is None:
        tData = {
            "username": USERNAME,
            "password": PASSWORD,
            "grant_type": "password",
            "scope": SCOPE,
            "takeExclusiveSignOnControl": "true"
        };
    else:
        tData = {
            "refresh_token": refreshToken,
            "grant_type": "refresh_token",
        };

    # Make a REST call to get latest access token
    response = requests.post(
        TOKEN_ENDPOINT,
        headers = {
            "Accept": "application/json"
        },
        data = tData,
        auth = (
            CLIENT_ID,
            CLIENT_SECRET
        )
    )
    
    if response.status_code != 200:
        raise Exception("Failed to get access token {0} - {1}".format(response.status_code, response.text));

    # Return the new token
    return json.loads(response.text);

def saveToken(tknObject):
    tf = open(TOKEN_FILE, "w+");
    print("Saving the new token");
    # Append the expiry time to token
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10;
    # Store it in the file
    json.dump(tknObject, tf, indent=4)
    
def getToken():
    try:
        print("Reading the token from: " + TOKEN_FILE);
        # Read the token from a file
        tf = open(TOKEN_FILE, "r+")
        tknObject = json.load(tf);

        # Is access token valid
        if tknObject["expiry_tm"] > time.time():
            # return access token
            return tknObject["access_token"];

        print("Token expired, refreshing a new one...");
        tf.close();
        # Get a new token from refresh token
        tknObject = _requestNewToken(tknObject["refresh_token"]);

    except Exception as exp:
        print("Caught exception: " + str(exp))
        print("Getting a new token using Password Grant...");
        tknObject = _requestNewToken(None);

    # Persist this token for future queries
    saveToken(tknObject)
#    print("Token is: " + tknObject["access_token"])
    # Return access token
    return tknObject["access_token"];

accessToken = getToken();
print("Have token now");
print("Token is: " + accessToken)

Reading the token from: token.txt
Token expired, refreshing a new one...
Caught exception: Failed to get access token 400 - {"error":"invalid_grant"   } 
Getting a new token using Password Grant...
Saving the new token
Have token now
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwic1d4eld3S0ppSmVIVzNvLUlaU1drVURPS1hUTDdPeW5Fam9lbHJmbk12YUZZcUhsRVcyV0tOaFVjeVFQa3hMY3BJNnFuWmVsRGVBS0I4MXZDenNMRnZES3pMR3lydEZ1aUVMZFY1clQxUFRPTEJ1Z0RvVXBhaHJkOXRyeHFaSnMtRjc3a2s3MUk5VktJOTdHaWxxTXhTRXVuczdNR0pPc29fc0lWcWJaR2x0czZ5QnFCZVhPMWVjdm5YWV9vU09HeVN3R1oyNFpnWU1BTzNJYzExb2tHZEJab3VCSWJxUGRDeUM0VDZqeTYyZTJtMlNhTjQtbjVyR193ZmJ6TUM3cVBjaXdaSGRrQ1hOVVdqVl85ekpzWTBSMFJVRkZ5NW1pcTRKWW14dGVUNlhNSGZZaENtVkdaTzd3T2RoLWN1QjdBeFl5SUM0RzhvTWhDWkFmR1pjal9JTXpaTmxUeVYyeGdPazBVV2ZKOUZtMW9ORkxDeGVXQmZPM0UzRENMSjdPeDR1SUtSVy1pejBtQzRrRy1lYUVEVEliZDdGR1lnMjZFTjhQV1NvWERvaDFUVDNaOWliLUg2X1Q2UDJHQl9IRnZvQmJ3WmpjT

## Request Available ESG Bulk File Sets

The purpose of ESG bulk service is obtaining ESG content in bulk.  The content is available as:

* A full JSON data file containing history for all measures and all organizations.
* A delta JSON data file that contains only incremental changes to the universe since last week. 

A customer can examine the available File Sets and is expected to:

* Build the initial ESG representation with the full files
* Apply delta, changes, as they become available
* Fill a gap if the retrival was not completed and the content missed remains available

This step can additionally be used to verify the availability and type of the file, for example:
* ESGRawFullScheme
* ESGScoresFull
* ESGScoresWealthFull

In [5]:
FILESET_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/file-sets?bucket='+ RDP_ESG_BUCKET

def requestFileSets(token, withNext, skipToken, attributes):   
    global FILESET_ENDPOINT
     
    
    print("Obtaining FileSets in ESG Bucket...")
  
    FILESET_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/file-sets?bucket='+ RDP_ESG_BUCKET
    
    querystring = {}
    payload = ""
    jsonfull = ""
    jsonpartial = ""
    
    headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + token,
            'cache-control': "no-cache"
    }

    if attributes:
        FILESET_ENDPOINT = FILESET_ENDPOINT + attributes
    if withNext:
        FILESET_ENDPOINT = FILESET_ENDPOINT + '&skipToken=' +skipToken
    
    print('GET '+FILESET_ENDPOINT )    
    response = requests.request("GET", FILESET_ENDPOINT, data=payload, headers=headers, params=querystring)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET", FILESET_ENDPOINT, data=payload, headers=headers, params=querystring)
         
    print('Ra401f-e870-2a782517-bf69-a887c7894fb9w response=');
    print(response);
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)        
        return jsonFullResp; 
    else:
        return '';

jsonFullResp = requestFileSets(accessToken, False, '','');

print('Parsed json response=');
print(json.dumps(jsonFullResp, indent=2));
print('Same response, tabular view');
df = pd.json_normalize(jsonFullResp['value'])
df

Obtaining FileSets in ESG Bucket...
GET https://api.refinitiv.com/file-store/v1/file-sets?bucket=bulk-esg
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4017-5af4-dee17d3f-8e8c-f60b5c07d995",
      "name": "Bulk-ESG-Global-Symbology-Cusip-v1-Jsonl-Init-2022-09-25T10:31:39.592Z",
      "bucketName": "bulk-ESG",
      "packageId": "4a7b-19e3-ac9446f0-9db9-9eab901a4953",
      "attributes": [
        {
          "name": "ContentType",
          "value": "Symbology Cusip"
        }
      ],
      "files": [
        "4d57-c774-e34d6032-9f39-0eb0d77a132e"
      ],
      "numFiles": 1,
      "contentFrom": "1970-01-01T00:00:00Z",
      "contentTo": "2022-09-25T10:25:00Z",
      "availableFrom": "2022-09-25T10:33:01Z",
      "availableTo": "2022-10-25T10:33:01Z",
      "status": "READY",
      "created": "2022-09-25T10:33:01Z",
      "modified": "2022-09-25T10:33:07Z"
    },
    {
      "id": "4030-252f-dcbb1ef0-9016-1ae9a9c30c01",
      "name": "Bulk-ES

,id,name,bucketName,packageId,attributes,files,numFiles,contentFrom,contentTo,availableFrom,availableTo,status,created,modified
0,4017-5af4-dee17d3f-8e8c-f60b5c07d995,Bulk-ESG-Global-Symbology-Cusip-v1-Jsonl-Init-...,bulk-ESG,4a7b-19e3-ac9446f0-9db9-9eab901a4953,"[{'name': 'ContentType', 'value': 'Symbology C...",[4d57-c774-e34d6032-9f39-0eb0d77a132e],1,1970-01-01T00:00:00Z,2022-09-25T10:25:00Z,2022-09-25T10:33:01Z,2022-10-25T10:33:01Z,READY,2022-09-25T10:33:01Z,2022-09-25T10:33:07Z
1,4030-252f-dcbb1ef0-9016-1ae9a9c30c01,Bulk-ESG-Global-Symbology-ISIN-v1-Jsonl-Delta-...,bulk-ESG,42db-71ff-ea4bd04c-9739-849990675c7d,"[{'name': 'ContentType', 'value': 'Symbology I...",[4d9e-4238-c76ddd60-a570-f2b59f8912ab],1,2022-09-25T10:25:00Z,2022-10-02T10:25:00Z,2022-10-02T10:33:33Z,2022-10-16T10:33:32Z,READY,2022-10-02T10:33:33Z,2022-10-02T10:33:34Z
2,4047-2c92-3234778e-bc70-3ded16bb6894,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Env-Jso...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4c17-f73c-27d571a8-a048-1e86c57e63b5],1,2022-09-18T17:35:00Z,2022-09-25T17:35:00Z,2022-09-25T17:46:34Z,2022-10-09T17:46:33Z,READY,2022-09-25T17:46:34Z,2022-09-25T17:49:20Z
3,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Csv...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[416a-9501-e8953755-ab8a-fcd07740c6a6, 442d-67...",7,1970-01-01T00:00:00Z,2022-10-02T16:05:00Z,2022-10-02T16:24:30Z,2022-10-16T16:24:30Z,READY,2022-10-02T16:24:30Z,2022-10-02T16:52:15Z
4,408d-b75f-fa8dca55-8fb1-0664bca59b74,Bulk-ESG-Global-Symbology-SEDOL-v1-Jsonl-Init-...,bulk-ESG,48ff-7e86-20bd08b2-8bbf-ee1b93395d59,"[{'name': 'ContentType', 'value': 'Symbology S...",[4d2a-4d49-bfd26986-a842-e494d7ce8adc],1,1970-01-01T00:00:00Z,2022-09-11T10:25:00Z,2022-09-11T10:41:22Z,2022-10-11T10:41:22Z,READY,2022-09-11T10:41:22Z,2022-09-11T10:41:31Z
5,408e-eb7b-6de21997-aa98-7d3bb87fe1e8,Bulk-ESG-Global-Symbology-Cusip-v1-Jsonl-Delta...,bulk-ESG,442c-d526-65702a11-b6b2-54f7d4f3cd7f,"[{'name': 'ContentType', 'value': 'Symbology C...",[4f15-8b23-62cff96c-bcce-c5a70a155248],1,2022-09-18T10:25:00Z,2022-09-25T10:25:00Z,2022-09-25T10:33:09Z,2022-10-09T10:33:09Z,READY,2022-09-25T10:33:09Z,2022-09-25T10:33:11Z
6,40a1-9428-2fd2d4f0-bc97-ffbdd310fa12,Bulk-ESG-Global-Raw-Current-Limited-SchemeB-v1...,bulk-ESG,448b-4cc8-301b4e6a-9f6e-a0935eb8454b,"[{'name': 'ContentType', 'value': 'ESG Raw Cur...",[4aea-0657-3db1d89f-a32b-dd6b6d136eef],1,2022-09-18T17:35:00Z,2022-09-25T17:35:00Z,2022-09-25T17:42:04Z,2022-10-09T17:42:03Z,READY,2022-09-25T17:42:04Z,2022-09-25T17:42:19Z
7,40a3-4364-66a9dfd6-975a-466d7ae2eaf8,Bulk-ESG-Global-Scores-Full-v1-Csv-Delta-2022-...,bulk-ESG,4f1d-c1e4-b448e23b-a51a-e4315800de31,"[{'name': 'ContentType', 'value': 'ESG Scores'}]",[4a14-212d-8a987499-b834-799de966790b],1,2022-09-18T16:25:00Z,2022-09-25T16:25:00Z,2022-09-25T16:34:07Z,2022-10-09T16:34:07Z,READY,2022-09-25T16:34:07Z,2022-09-25T16:34:26Z
8,40c2-b143-7ecaea6a-89d8-73cbe4a1ecd5,Bulk-ESG-Global-Raw-Current-Limited-SchemeB-v1...,bulk-ESG,448b-4cc8-301b4e6a-9f6e-a0935eb8454b,"[{'name': 'ContentType', 'value': 'ESG Raw Cur...",[459e-a0f9-b97e255e-b43b-9ead05bced55],1,1970-01-01T00:00:00Z,2022-09-25T17:35:00Z,2022-09-25T17:43:48Z,2022-10-09T17:43:47Z,READY,2022-09-25T17:43:48Z,2022-09-25T17:44:31Z
9,40cc-9a94-5bcb8210-9c92-e730f23fa1bb,Bulk-ESG-Global-Raw-Full-Limited-SchemeB-v1-So...,bulk-ESG,464a-f34b-3b6fb3ba-8db1-1489b6638963,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4fa3-0835-8a1e27cf-b4db-f0e6301c73ce],1,1970-01-01T00:00:00Z,2022-09-25T16:05:00Z,2022-09-25T16:14:26Z,2022-10-09T16:14:26Z,READY,2022-09-25T16:14:26Z,2022-09-25T16:17:26Z


### Select ESG FileSet Id

In [13]:
# Optionally, copy from result of "Present FileSet Results in Tabular View"
FILESET_ID = input()
print('FILESET_ID selected is: ' + FILESET_ID)

 4081-6dd8-09c497f8-bdf5-66a9b303b0c4


FILESET_ID selected is: 4081-6dd8-09c497f8-bdf5-66a9b303b0c4


### Select ESG PackageId

In [10]:
# Optionally, copy from result of "Present FileSet Results in Tabular View"
PACKAGE_ID = input()
print('PACKAGE_ID selected is: ' + PACKAGE_ID)

 47c5-675c-8e5962b4-af5c-10646366c5e9


PACKAGE_ID selected is: 47c5-675c-8e5962b4-af5c-10646366c5e9


### Paginate Through the Available FileSets
(interrupt at any point)

In [8]:
i = 1
while "@nextLink" in jsonFullResp: 
    print('<<< Iteraction: '+str(i)+' >>>  More exists: '+ jsonFullResp['@nextLink'] + ', skipToken is: ' + jsonFullResp['@nextLink'][-62:]+'\n')
    jsonFullResp = requestFileSets(accessToken, True, jsonFullResp['@nextLink'][-62:],'');
    print(json.dumps(jsonFullResp, indent=2));
    i+=1;
print('Last response without next=');
print(json.dumps(jsonFullResp, indent=2));

<<< Iteraction: 1 >>>  More exists: /file-store/v1/file-sets?bucket=ESG&pageSize=25&skipToken=ZmlsZXNldElkPTQ0ZGMtMmVmYS1iMTk0YjczZS04MDFjLTNhYWIyM2ZlMjExNg, skipToken is: ZmlsZXNldElkPTQ0ZGMtMmVmYS1iMTk0YjczZS04MDFjLTNhYWIyM2ZlMjExNg

Obtaining FileSets in ESG Bucket...
GET https://api.refinitiv.com/file-store/v1/file-sets?bucket=ESG&skipToken=ZmlsZXNldElkPTQ0ZGMtMmVmYS1iMTk0YjczZS04MDFjLTNhYWIyM2ZlMjExNg
Reading the token from: token.txt
Token expired, refreshing a new one...
Caught exception: Failed to get access token 400 - {"error":"invalid_grant"   } 
Getting a new token using Password Grant...
Saving the new token
Raw response=
<Response [200]>
{
  "value": [
    {
      "id": "44ed-3123-5c76e7b4-abe3-efb6951256f8",
      "name": "RFT-ESG-Raw-Current-SchemeA-Delta-2022-03-13",
      "bucketName": "ESG",
      "packageId": "4c62-b05c-2a529a9d-81b3-224eacd50379",
      "attributes": [
        {
          "name": "ContentType",
          "value": "ESG Raw Current A"
        },
    

KeyboardInterrupt: 

### Retrieve FileSets of Specific File Type (Filter By Attribute and By PackageId)
The file types may change over time, at the time of this writing, the available FileSets are of types:

* ESG Raw Full A
* ESG Raw Full B
* ESG Raw Current A
* ESG Raw Current B
* ESG Sources
* ESG Raw Wealth Standard

* Symbology Cusip
* Symbology SEDOL
* Symbology Organization
* Symbology Instrument Quote

Further, the selected package, if also filtering by packageId has to contain the files per filtering arrtibutes, in order to request their listing succefully,
otherwise the result will be empty.


In [8]:
jsonFullResp = requestFileSets(accessToken, False, '','&ContentType:ESG Raw Current A');
print('Parsed json response=');
print(json.dumps(jsonFullResp, indent=2));
print('Same response, tabular view');
df = pd.json_normalize(jsonFullResp['value'])
df

Obtaining FileSets in ESG Bucket...
GET https://api.refinitiv.com/file-store/v1/file-sets?bucket=bulk-esg&ContentType:ESG Raw Current A
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4017-5af4-dee17d3f-8e8c-f60b5c07d995",
      "name": "Bulk-ESG-Global-Symbology-Cusip-v1-Jsonl-Init-2022-09-25T10:31:39.592Z",
      "bucketName": "bulk-ESG",
      "packageId": "4a7b-19e3-ac9446f0-9db9-9eab901a4953",
      "attributes": [
        {
          "name": "ContentType",
          "value": "Symbology Cusip"
        }
      ],
      "files": [
        "4d57-c774-e34d6032-9f39-0eb0d77a132e"
      ],
      "numFiles": 1,
      "contentFrom": "1970-01-01T00:00:00Z",
      "contentTo": "2022-09-25T10:25:00Z",
      "availableFrom": "2022-09-25T10:33:01Z",
      "availableTo": "2022-10-25T10:33:01Z",
      "status": "READY",
      "created": "2022-09-25T10:33:01Z",
      "modified": "2022-09-25T10:33:07Z"
    },
    {
      "id": "4030-252f-dcbb1ef0-9016-1ae9a9c

,id,name,bucketName,packageId,attributes,files,numFiles,contentFrom,contentTo,availableFrom,availableTo,status,created,modified
0,4017-5af4-dee17d3f-8e8c-f60b5c07d995,Bulk-ESG-Global-Symbology-Cusip-v1-Jsonl-Init-...,bulk-ESG,4a7b-19e3-ac9446f0-9db9-9eab901a4953,"[{'name': 'ContentType', 'value': 'Symbology C...",[4d57-c774-e34d6032-9f39-0eb0d77a132e],1,1970-01-01T00:00:00Z,2022-09-25T10:25:00Z,2022-09-25T10:33:01Z,2022-10-25T10:33:01Z,READY,2022-09-25T10:33:01Z,2022-09-25T10:33:07Z
1,4030-252f-dcbb1ef0-9016-1ae9a9c30c01,Bulk-ESG-Global-Symbology-ISIN-v1-Jsonl-Delta-...,bulk-ESG,42db-71ff-ea4bd04c-9739-849990675c7d,"[{'name': 'ContentType', 'value': 'Symbology I...",[4d9e-4238-c76ddd60-a570-f2b59f8912ab],1,2022-09-25T10:25:00Z,2022-10-02T10:25:00Z,2022-10-02T10:33:33Z,2022-10-16T10:33:32Z,READY,2022-10-02T10:33:33Z,2022-10-02T10:33:34Z
2,4047-2c92-3234778e-bc70-3ded16bb6894,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Env-Jso...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4c17-f73c-27d571a8-a048-1e86c57e63b5],1,2022-09-18T17:35:00Z,2022-09-25T17:35:00Z,2022-09-25T17:46:34Z,2022-10-09T17:46:33Z,READY,2022-09-25T17:46:34Z,2022-09-25T17:49:20Z
3,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Csv...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[416a-9501-e8953755-ab8a-fcd07740c6a6, 442d-67...",7,1970-01-01T00:00:00Z,2022-10-02T16:05:00Z,2022-10-02T16:24:30Z,2022-10-16T16:24:30Z,READY,2022-10-02T16:24:30Z,2022-10-02T16:52:15Z
4,408d-b75f-fa8dca55-8fb1-0664bca59b74,Bulk-ESG-Global-Symbology-SEDOL-v1-Jsonl-Init-...,bulk-ESG,48ff-7e86-20bd08b2-8bbf-ee1b93395d59,"[{'name': 'ContentType', 'value': 'Symbology S...",[4d2a-4d49-bfd26986-a842-e494d7ce8adc],1,1970-01-01T00:00:00Z,2022-09-11T10:25:00Z,2022-09-11T10:41:22Z,2022-10-11T10:41:22Z,READY,2022-09-11T10:41:22Z,2022-09-11T10:41:31Z
5,408e-eb7b-6de21997-aa98-7d3bb87fe1e8,Bulk-ESG-Global-Symbology-Cusip-v1-Jsonl-Delta...,bulk-ESG,442c-d526-65702a11-b6b2-54f7d4f3cd7f,"[{'name': 'ContentType', 'value': 'Symbology C...",[4f15-8b23-62cff96c-bcce-c5a70a155248],1,2022-09-18T10:25:00Z,2022-09-25T10:25:00Z,2022-09-25T10:33:09Z,2022-10-09T10:33:09Z,READY,2022-09-25T10:33:09Z,2022-09-25T10:33:11Z
6,40a1-9428-2fd2d4f0-bc97-ffbdd310fa12,Bulk-ESG-Global-Raw-Current-Limited-SchemeB-v1...,bulk-ESG,448b-4cc8-301b4e6a-9f6e-a0935eb8454b,"[{'name': 'ContentType', 'value': 'ESG Raw Cur...",[4aea-0657-3db1d89f-a32b-dd6b6d136eef],1,2022-09-18T17:35:00Z,2022-09-25T17:35:00Z,2022-09-25T17:42:04Z,2022-10-09T17:42:03Z,READY,2022-09-25T17:42:04Z,2022-09-25T17:42:19Z
7,40a3-4364-66a9dfd6-975a-466d7ae2eaf8,Bulk-ESG-Global-Scores-Full-v1-Csv-Delta-2022-...,bulk-ESG,4f1d-c1e4-b448e23b-a51a-e4315800de31,"[{'name': 'ContentType', 'value': 'ESG Scores'}]",[4a14-212d-8a987499-b834-799de966790b],1,2022-09-18T16:25:00Z,2022-09-25T16:25:00Z,2022-09-25T16:34:07Z,2022-10-09T16:34:07Z,READY,2022-09-25T16:34:07Z,2022-09-25T16:34:26Z
8,40c2-b143-7ecaea6a-89d8-73cbe4a1ecd5,Bulk-ESG-Global-Raw-Current-Limited-SchemeB-v1...,bulk-ESG,448b-4cc8-301b4e6a-9f6e-a0935eb8454b,"[{'name': 'ContentType', 'value': 'ESG Raw Cur...",[459e-a0f9-b97e255e-b43b-9ead05bced55],1,1970-01-01T00:00:00Z,2022-09-25T17:35:00Z,2022-09-25T17:43:48Z,2022-10-09T17:43:47Z,READY,2022-09-25T17:43:48Z,2022-09-25T17:44:31Z
9,40cc-9a94-5bcb8210-9c92-e730f23fa1bb,Bulk-ESG-Global-Raw-Full-Limited-SchemeB-v1-So...,bulk-ESG,464a-f34b-3b6fb3ba-8db1-1489b6638963,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4fa3-0835-8a1e27cf-b4db-f0e6301c73ce],1,1970-01-01T00:00:00Z,2022-09-25T16:05:00Z,2022-09-25T16:14:26Z,2022-10-09T16:14:26Z,READY,2022-09-25T16:14:26Z,2022-09-25T16:17:26Z


In [ ]:
#jsonFullResp = requestFileSets(accessToken, False, '','&attributes=ContentType:Symbology Cusip');
jsonFullResp = requestFileSets(accessToken, False, '','&attributes=ContentType:ESG Scores');
#jsonFullResp = requestFileSets(accessToken, False, '','&attributes=ContentType:Symbology SEDOL');
print('Parsed json response=');
print(json.dumps(jsonFullResp, indent=2));
print('Same response, tabular view');
df = pd.json_normalize(jsonFullResp['value'])
df

In [11]:
print('Requesting for packageId= '+PACKAGE_ID)
jsonFullResp = requestFileSets(accessToken, False, '','&packageId='+PACKAGE_ID); #+'&attributes=ContentType:ESG Sources');
print('Parsed json response=');
print(json.dumps(jsonFullResp, indent=2));
print('Same response, tabular view');
df = pd.json_normalize(jsonFullResp['value'])
df

Requesting for packageId= 47c5-675c-8e5962b4-af5c-10646366c5e9
Obtaining FileSets in ESG Bucket...
GET https://api.refinitiv.com/file-store/v1/file-sets?bucket=bulk-esg&packageId=47c5-675c-8e5962b4-af5c-10646366c5e9
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4047-2c92-3234778e-bc70-3ded16bb6894",
      "name": "Bulk-ESG-Global-Raw-Current-SchemeB-v1-Env-Jsonl-Delta-2022-09-25T17:41:48.050Z",
      "bucketName": "bulk-ESG",
      "packageId": "47c5-675c-8e5962b4-af5c-10646366c5e9",
      "attributes": [
        {
          "name": "ContentType",
          "value": "ESG Raw Full B"
        }
      ],
      "files": [
        "4c17-f73c-27d571a8-a048-1e86c57e63b5"
      ],
      "numFiles": 1,
      "contentFrom": "2022-09-18T17:35:00Z",
      "contentTo": "2022-09-25T17:35:00Z",
      "availableFrom": "2022-09-25T17:46:34Z",
      "availableTo": "2022-10-09T17:46:33Z",
      "status": "READY",
      "created": "2022-09-25T17:46:34Z",
      "mod

,id,name,bucketName,packageId,attributes,files,numFiles,contentFrom,contentTo,availableFrom,availableTo,status,created,modified
0,4047-2c92-3234778e-bc70-3ded16bb6894,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Env-Jso...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4c17-f73c-27d571a8-a048-1e86c57e63b5],1,2022-09-18T17:35:00Z,2022-09-25T17:35:00Z,2022-09-25T17:46:34Z,2022-10-09T17:46:33Z,READY,2022-09-25T17:46:34Z,2022-09-25T17:49:20Z
1,40dd-8cf0-70e5f03f-a68c-0115e69c7f1f,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Gov-Csv...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4ebd-0ff8-905e7971-aef5-3a4e7e8c26af],1,2022-09-25T17:35:00Z,2022-10-02T17:35:00Z,2022-10-02T17:48:11Z,2022-10-16T17:48:11Z,READY,2022-10-02T17:48:11Z,2022-10-02T17:48:48Z
2,4102-a1f4-ddb437eb-be9d-27a63ed77ec8,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Social-...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4d24-3a79-7124cefa-8c7d-20e040c82e8c],1,2022-09-18T17:35:00Z,2022-09-25T17:35:00Z,2022-09-25T17:44:42Z,2022-10-09T17:44:42Z,READY,2022-09-25T17:44:42Z,2022-09-25T17:49:20Z
3,4135-290d-f7bd6010-be8d-f58f79ba4a65,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Social-...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4fda-b3ba-ba9fa72f-8ec3-a1a4de0643a0],1,2022-09-18T17:35:00Z,2022-09-25T17:35:00Z,2022-09-25T17:43:57Z,2022-10-09T17:43:57Z,READY,2022-09-25T17:43:57Z,2022-09-25T17:49:19Z
4,4153-cd91-1a3138f3-8395-9559f30d1f1d,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Gov-Csv...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[484f-c34e-ce71af45-ac43-fc3a2edf9d2e],1,2022-09-18T17:35:00Z,2022-09-25T17:35:00Z,2022-09-25T17:48:43Z,2022-10-09T17:48:43Z,READY,2022-09-25T17:48:43Z,2022-09-25T17:49:20Z
5,4169-6136-9cdfacbb-94d5-77de4b38487b,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Social-...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[405d-a17d-0ae46596-aefc-b1c86bcdaf76],1,2022-09-25T17:35:00Z,2022-10-02T17:35:00Z,2022-10-02T17:44:01Z,2022-10-16T17:44:01Z,READY,2022-10-02T17:44:01Z,2022-10-02T17:48:47Z
6,43a2-1828-f61c0c02-86d4-bbf6e5ebd8fd,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Social-...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4cf4-5aae-62c0eb26-9844-a92f2f33e78b],1,1970-01-01T00:00:00Z,2022-10-02T17:35:00Z,2022-10-02T17:44:27Z,2022-10-16T17:44:27Z,READY,2022-10-02T17:44:27Z,2022-10-02T17:50:08Z
7,44f4-e636-2c33b438-9c74-e0fbc7880181,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Gov-Jso...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4332-ffba-e2c9c888-b81d-1cd2a00e4c37],1,2022-09-25T17:35:00Z,2022-10-02T17:35:00Z,2022-10-02T17:47:46Z,2022-10-16T17:47:46Z,READY,2022-10-02T17:47:46Z,2022-10-02T17:48:48Z
8,466a-ca4e-7503869d-b025-2ed0e692dc91,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Env-Jso...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4eb1-c602-2c8f1469-a061-fc6652cbd41d],1,2022-09-25T17:35:00Z,2022-10-02T17:35:00Z,2022-10-02T17:45:58Z,2022-10-16T17:45:58Z,READY,2022-10-02T17:45:58Z,2022-10-02T17:48:48Z
9,4671-5c4d-b2366a26-8055-d8043a9e7b78,Bulk-ESG-Global-Raw-Current-SchemeB-v1-Env-Jso...,bulk-ESG,47c5-675c-8e5962b4-af5c-10646366c5e9,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[41a1-315c-b837a823-a955-83c01efc88ea],1,1970-01-01T00:00:00Z,2022-10-02T17:35:00Z,2022-10-02T17:47:08Z,2022-10-16T17:47:08Z,READY,2022-10-02T17:47:08Z,2022-10-02T17:50:09Z


In [9]:
#jsonFullResp = requestFileSets(accessToken, False, '','&attributes=ContentType:Symbology Cusip');
jsonFullResp = requestFileSets(accessToken, False, '','&attributes=ContentType:ESG Scores&contentFrom:2021-05-17T00:00:00Z&contentTo:2021-05-18T00:00:00Z');
#jsonFullResp = requestFileSets(accessToken, False, '','&attributes=ContentType:Symbology SEDOL');
print('Parsed json response=');
print(json.dumps(jsonFullResp, indent=2));
print('Same response, tabular view');
df = pd.json_normalize(jsonFullResp['value'])
df

Obtaining FileSets in ESG Bucket...
GET https://api.refinitiv.com/file-store/v1/file-sets?bucket=ESG&attributes=ContentType:ESG Scores&contentFrom:2021-05-17T00:00:00Z&contentTo:2021-05-18T00:00:00Z
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4075-697d-32ddf0b5-89f5-81f8b2a58f00",
      "name": "RFT-ESG-Scores-Wealth-Standard-init-2022-05-01",
      "bucketName": "ESG",
      "packageId": "4bcc-4602-0a57ebb2-baf0-1fc9825e76b6",
      "attributes": [
        {
          "name": "ContentType",
          "value": "ESG Scores"
        },
        {
          "name": "ResultCount",
          "value": "34537"
        }
      ],
      "files": [
        "4b2e-60ed-7062ab63-9fa9-de261ee119c9"
      ],
      "numFiles": 1,
      "contentFrom": "1970-01-01T00:00:00Z",
      "contentTo": "2022-05-01T17:15:00Z",
      "availableFrom": "2022-05-01T17:23:19Z",
      "availableTo": "2022-06-01T17:23:19Z",
      "status": "READY",
      "created": "2022-05-01T

,id,name,bucketName,packageId,attributes,files,numFiles,contentFrom,contentTo,availableFrom,availableTo,status,created,modified
0,4075-697d-32ddf0b5-89f5-81f8b2a58f00,RFT-ESG-Scores-Wealth-Standard-init-2022-05-01,ESG,4bcc-4602-0a57ebb2-baf0-1fc9825e76b6,"[{'name': 'ContentType', 'value': 'ESG Scores'...",[4b2e-60ed-7062ab63-9fa9-de261ee119c9],1,1970-01-01T00:00:00Z,2022-05-01T17:15:00Z,2022-05-01T17:23:19Z,2022-06-01T17:23:19Z,READY,2022-05-01T17:23:19Z,2022-05-01T17:23:26Z
1,40ff-8971-2ed23ff1-b760-8707418aa010,RFT-ESG-Scores-Current-init-2022-04-24,ESG,4157-5865-69f2417d-8b77-21a75f7f12ad,"[{'name': 'ResultCount', 'value': '12294'}, {'...",[44f0-bf5c-322803cf-a1bc-9ee753e11c27],1,1970-01-01T00:00:00Z,2022-04-24T17:05:00Z,2022-04-24T17:12:28Z,2022-05-24T17:12:28Z,READY,2022-04-24T17:12:28Z,2022-04-24T17:12:32Z
2,4210-8f2a-ea9de16a-9384-b81589052aaa,RFT-ESG-Scores-Current-init-2022-05-01,ESG,4157-5865-69f2417d-8b77-21a75f7f12ad,"[{'name': 'ResultCount', 'value': '12328'}, {'...",[4f90-3b6a-34e17f3b-bb55-c864efa960e3],1,1970-01-01T00:00:00Z,2022-05-01T17:05:00Z,2022-05-01T17:12:37Z,2022-06-01T17:12:36Z,READY,2022-05-01T17:12:37Z,2022-05-01T17:12:40Z
3,4233-2ab4-839f956f-9d7f-64d836c864b9,RFT-ESG-Scores-Wealth-Standard-delta-2022-04-10,ESG,4bcc-4602-0a57ebb2-baf0-1fc9825e76b6,"[{'name': 'ResultCount', 'value': '28341'}, {'...",[480b-3ef0-d2dd680e-be89-dfdbca4cbfa9],1,2022-04-03T17:15:00Z,2022-04-10T17:15:00Z,2022-04-10T17:22:43Z,2022-05-10T17:22:43Z,READY,2022-04-10T17:22:43Z,2022-04-10T17:22:49Z
4,424e-af0f-4ee9982c-8e14-74c1ad6cecf0,RFT-ESG-Scores-Current-init-2022-04-10,ESG,4157-5865-69f2417d-8b77-21a75f7f12ad,"[{'name': 'ContentType', 'value': 'ESG Scores'...",[4837-2b0f-fc94f5ce-aaae-1dd2979cdff0],1,1970-01-01T00:00:00Z,2022-04-10T17:05:00Z,2022-04-10T17:12:16Z,2022-05-10T17:12:16Z,READY,2022-04-10T17:12:16Z,2022-04-10T17:12:20Z
5,42b6-0938-e939678c-8774-447bcd407bf0,RFT-ESG-Scores-Wealth-Standard-init-2022-04-24,ESG,4bcc-4602-0a57ebb2-baf0-1fc9825e76b6,"[{'name': 'ResultCount', 'value': '33016'}, {'...",[4736-2f4e-8f285b77-8c36-64a26ebd5769],1,1970-01-01T00:00:00Z,2022-04-24T17:15:00Z,2022-04-24T17:22:51Z,2022-05-24T17:22:51Z,READY,2022-04-24T17:22:51Z,2022-04-24T17:22:57Z
6,443b-fb14-579a962b-97e2-dc93ed9aa96b,RFT-ESG-Scores-Wealth-Standard-delta-2022-04-24,ESG,4bcc-4602-0a57ebb2-baf0-1fc9825e76b6,"[{'name': 'ContentType', 'value': 'ESG Scores'...",[4b37-6f39-c4a2202a-8bd5-d9e35c78ed84],1,2022-04-17T17:15:00Z,2022-04-24T17:15:00Z,2022-04-24T17:23:07Z,2022-05-24T17:23:07Z,READY,2022-04-24T17:23:07Z,2022-04-24T17:23:13Z
7,4474-23d1-7d15d4fe-a26c-6630856065f5,RFT-ESG-Scores-Current-delta-2022-04-17,ESG,4157-5865-69f2417d-8b77-21a75f7f12ad,"[{'name': 'ResultCount', 'value': '10871'}, {'...",[406b-15d0-339ebe78-ab38-b56cfd7ae8f2],1,2022-04-10T17:05:00Z,2022-04-17T17:05:00Z,2022-04-17T17:12:40Z,2022-05-17T17:12:40Z,READY,2022-04-17T17:12:40Z,2022-04-17T17:12:44Z
8,449d-65ee-a9dbb31e-8f7e-24f1173bf075,RFT-ESG-Scores-Wealth-Full-Delta-2022-05-01,ESG,4f77-627a-c467ff19-9a21-8028baeeb2f4,"[{'name': 'ContentType', 'value': 'ESG Scores'...",[49ed-eb8a-1f480943-8abd-077ad2b2dee5],1,2022-04-24T16:35:00Z,2022-05-01T16:35:00Z,2022-05-01T16:43:17Z,2022-05-15T16:43:17Z,READY,2022-05-01T16:43:17Z,2022-05-01T16:43:26Z
9,45cc-4577-459f47a5-b0fe-64285bf90c9c,RFT-ESG-Scores-Wealth-Full-Init-2022-05-01,ESG,4f77-627a-c467ff19-9a21-8028baeeb2f4,"[{'name': 'ContentType', 'value': 'ESG Scores'...",[418e-5b75-a4831ac9-9945-a94ef140cf5b],1,1970-01-01T00:00:00Z,2022-05-01T16:35:00Z,2022-05-01T16:44:15Z,2022-05-15T16:44:14Z,READY,2022-05-01T16:44:15Z,2022-05-01T16:44:31Z


### Retrieve Complete File Details of FileSet ID

In [14]:
FILES_ENDPOINT_START = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files?filesetId='
 
def requestFileDetails(token, fileSetId):   

    print("Obtaining File details for FileSet= "+ fileSetId + " ...")
    print("(If result is Response=400, make sure that fileSetId is set with a valid value...)")
    FILES_ENDPOINT = FILES_ENDPOINT_START + fileSetId
  
    querystring = {}
    payload = ""
    jsonfull = ""
    jsonpartial = ""
    
    headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + token,
            'cache-control': "no-cache"
    }
        
    response = requests.request("GET", FILES_ENDPOINT, data=payload, headers=headers, params=querystring)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET", FILES_ENDPOINT, data=payload, headers=headers, params=querystring)
         
    print('Raw response=');
    print(response);
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)        
        return jsonFullResp; 
    else:
        return '';

jsonFullResp = requestFileDetails(accessToken, FILESET_ID);

print('Parsed json response=');
print(json.dumps(jsonFullResp, indent=2));
df = pd.json_normalize(jsonFullResp['value'])
df

Obtaining File details for FileSet= 4081-6dd8-09c497f8-bdf5-66a9b303b0c4 ...
(If result is Response=400, make sure that fileSetId is set with a valid value...)
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "416a-9501-e8953755-ab8a-fcd07740c6a6",
      "filename": "Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16:11:18.777Z-part4.csv.gz",
      "filesetId": "4081-6dd8-09c497f8-bdf5-66a9b303b0c4",
      "storageLocation": {
        "url": "https://a206464-bulk-esg.s3.amazonaws.com/Bulk-ESG-Global-Raw-Full-SchemeB-v1/2022/10/02/Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16%3A11%3A18.777Z-part4.csv.gz",
        "@type": "s3"
      },
      "created": "2022-10-02T16:31:33Z",
      "modified": "2022-10-02T16:31:33Z",
      "href": "https://api.refinitiv.com/file-store/v1/files/416a-9501-e8953755-ab8a-fcd07740c6a6/stream",
      "fileSizeInBytes": 93023551,
      "md5": "4238bc2278d9ba16d4982542e28b8b31"
    },
    {
      "

,id,filename,filesetId,created,modified,href,fileSizeInBytes,md5,storageLocation.url,storageLocation.@type
0,416a-9501-e8953755-ab8a-fcd07740c6a6,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Ini...,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,2022-10-02T16:31:33Z,2022-10-02T16:31:33Z,https://api.refinitiv.com/file-store/v1/files/...,93023551,4238bc2278d9ba16d4982542e28b8b31,https://a206464-bulk-esg.s3.amazonaws.com/Bulk...,s3
1,442d-6769-ccc594a9-b06d-a79c0f2daa2e,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Ini...,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,2022-10-02T16:31:33Z,2022-10-02T16:31:33Z,https://api.refinitiv.com/file-store/v1/files/...,93131702,25a67a58558f4e6f359939fb9bb55be9,https://a206464-bulk-esg.s3.amazonaws.com/Bulk...,s3
2,48c1-536c-c98428dd-a599-18ec49ee9f12,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Ini...,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,2022-10-02T16:31:34Z,2022-10-02T16:31:34Z,https://api.refinitiv.com/file-store/v1/files/...,93148561,41d16e9533a65f7f72693742aa5cc924,https://a206464-bulk-esg.s3.amazonaws.com/Bulk...,s3
3,49e8-5c61-065c3338-875c-bcef8b1eb7a3,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Ini...,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,2022-10-02T16:31:33Z,2022-10-02T16:31:33Z,https://api.refinitiv.com/file-store/v1/files/...,93245772,a9778fb3326599d92dfb19d15f712db2,https://a206464-bulk-esg.s3.amazonaws.com/Bulk...,s3
4,4b57-a046-61dc284b-87f6-6ed9066295d1,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Ini...,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,2022-10-02T16:31:34Z,2022-10-02T16:31:34Z,https://api.refinitiv.com/file-store/v1/files/...,93060812,7dbc3775465ef8f49ce2f802d6e65bee,https://a206464-bulk-esg.s3.amazonaws.com/Bulk...,s3
5,4d9b-9d04-71d40fca-8d85-09b744a1d0f9,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Ini...,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,2022-10-02T16:31:33Z,2022-10-02T16:31:33Z,https://api.refinitiv.com/file-store/v1/files/...,93244237,476df0034e7068bffdf21c7f0e5e4a43,https://a206464-bulk-esg.s3.amazonaws.com/Bulk...,s3
6,4e9f-19db-0d50c673-9e28-952b6eaddde9,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Ini...,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,2022-10-02T16:31:33Z,2022-10-02T16:31:33Z,https://api.refinitiv.com/file-store/v1/files/...,93078938,3adac6c3189e3059f9a95385e95c12de,https://a206464-bulk-esg.s3.amazonaws.com/Bulk...,s3


### Select ESG File Id and File Name

In [25]:
# Optionally, copy from result of "Retrieve Complete File Details of FileSet ID"
FILE_ID = input()
print('FILE_ID selected is: ' + FILE_ID)
FILE_NAME = input()
print('FILE_NAME selected is: ' + FILE_NAME)

 416a-9501-e8953755-ab8a-fcd07740c6a6


FILE_ID selected is: 416a-9501-e8953755-ab8a-fcd07740c6a6


 Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16.777Z-part4.csv.gz


FILE_NAME selected is: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16.777Z-part4.csv.gz


###  Stream File via File Id using Redirect

In [32]:
import shutil

FILES_STREAM_ENDPOINT_START = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files/'

# use valid values, obtained from the previous step
exampleFileId = '4edd-99af-da829f42-8ddd-07fabfcddca9'  
exampleFileName = 'RFT-ESG-Sources-Full-Init-2021-01-17-part07.jsonl.gz'

def requestFileDownload(token, fileId, fileName):   
    FILES_STREAM_ENDPOINT = FILES_STREAM_ENDPOINT_START + fileId+ '/stream'
    print("Obtaining File ... " + FILES_STREAM_ENDPOINT)
  
    chunk_size = 1000
    
    headers = {
            'Authorization': 'Bearer ' + token,
            'cache-control': "no-cache",
            'Accept': '*/*'
    }
        
    response = requests.request("GET", FILES_STREAM_ENDPOINT, headers=headers, stream=True, allow_redirects=True)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET",FILES_STREAM_ENDPOINT, headers=headers, stream=True, allow_redirects=True)

         
    print('Response code=' + str(response.status_code));
    
    # running on windows and colons are not allowed in filenames
    fileName = fileName.replace(":",".")
    if response.status_code == 200:
        print('Processing...')
        with open(fileName, 'wb') as fd:
            shutil.copyfileobj(response.raw, fd) 
        print('Look for gzipped file named: '+ fileName + ' in current directory')
        response.connection.close()
        
    return; 

# consider below an example only
#requestFileDownload(accessToken, exampleFileId, exampleFileName);
requestFileDownload(accessToken, FILE_ID, FILE_NAME);

Obtaining File ... https://api.refinitiv.com/file-store/v1/files/416a-9501-e8953755-ab8a-fcd07740c6a6/stream
Reading the token from: token.txt
Response code=200
Processing...
Look for gzipped file named: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16.777Z-part4.csv.gz in current directory


### Select the Latest ESG FileSets (Init and Delta) as of Last Sunday per PackageId

In [30]:
import datetime

# determine what date last Sunday was
d = datetime.datetime.today()
#print(d)
sun_offset = (d.weekday() - 6) % 7
sunday = d - datetime.timedelta(days=sun_offset)

# format Sunday date to ESG bulk current requirements
sunday = sunday.replace(hour=0, minute=0, second=0, microsecond=0)
sunday = str(sunday).replace(' 00:00:00', 'T00:00:00Z')
print("Last Sunday was on", sunday)

#PACKAGE_ID - replace with yours 
PACKAGE_ID = '4288-ebb6-93372235-acb2-89882a826af1'
ESG_FILESET_RESP = requestFileSets(accessToken, False, '','&packageId='+PACKAGE_ID+'&availableFrom='+ sunday);
print('Parsed json response=');
print(json.dumps(ESG_FILESET_RESP, indent=2));
# now ESG_FILESET_RESP contains the requisite FileSetIds
df = pd.json_normalize(ESG_FILESET_RESP['value'])
df

Last Sunday was on 2022-10-02T00:00:00Z
Obtaining FileSets in ESG Bucket...
GET https://api.refinitiv.com/file-store/v1/file-sets?bucket=bulk-esg&packageId=4288-ebb6-93372235-acb2-89882a826af1&availableFrom=2022-10-02T00:00:00Z
Reading the token from: token.txt
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4081-6dd8-09c497f8-bdf5-66a9b303b0c4",
      "name": "Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Csv-Init-2022-10-02T16:11:18.777Z",
      "bucketName": "bulk-ESG",
      "packageId": "4288-ebb6-93372235-acb2-89882a826af1",
      "attributes": [
        {
          "name": "ContentType",
          "value": "ESG Raw Full B"
        }
      ],
      "files": [
        "416a-9501-e8953755-ab8a-fcd07740c6a6",
        "442d-6769-ccc594a9-b06d-a79c0f2daa2e",
        "48c1-536c-c98428dd-a599-18ec49ee9f12",
        "49e8-5c61-065c3338-875c-bcef8b1eb7a3",
        "4b57-a046-61dc284b-87f6-6ed9066295d1",
        "4d9b-9d04-71d40fca-8d85-09b744a1d0f9",
  

,id,name,bucketName,packageId,attributes,files,numFiles,contentFrom,contentTo,availableFrom,availableTo,status,created,modified
0,4081-6dd8-09c497f8-bdf5-66a9b303b0c4,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Csv...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[416a-9501-e8953755-ab8a-fcd07740c6a6, 442d-67...",7,1970-01-01T00:00:00Z,2022-10-02T16:05:00Z,2022-10-02T16:24:30Z,2022-10-16T16:24:30Z,READY,2022-10-02T16:24:30Z,2022-10-02T16:52:15Z
1,42fe-e041-3d309d8a-9053-6401b15060f4,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Csv-In...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[4514-6a21-3811ca92-9123-cd5ac21ed024, 4739-0d...",5,1970-01-01T00:00:00Z,2022-10-02T16:05:00Z,2022-10-02T16:48:30Z,2022-10-16T16:48:29Z,READY,2022-10-02T16:48:30Z,2022-10-02T16:52:16Z
2,4309-605b-5035be51-bc8a-7f0b7fdc0dd4,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Csv-De...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[4153-3eac-9cd64126-9091-be8bba15a940, 4987-4b...",4,2022-09-25T16:05:00Z,2022-10-02T16:05:00Z,2022-10-02T16:46:48Z,2022-10-16T16:46:48Z,READY,2022-10-02T16:46:48Z,2022-10-02T16:50:27Z
3,4324-86a5-cf2a45ba-ae33-f2527f9ba48f,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Jso...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[468c-b6bc-21e4bdee-967f-126fea012a3f, 4eab-89...",2,2022-09-25T16:05:00Z,2022-10-02T16:05:00Z,2022-10-02T16:17:35Z,2022-10-16T16:17:35Z,READY,2022-10-02T16:17:35Z,2022-10-02T16:50:26Z
4,4457-6b69-47333a71-bb58-165e7bdec0e5,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Csv-In...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[401d-38e5-8ea71a92-ac6d-70299f495ac3, 4122-a2...",5,1970-01-01T00:00:00Z,2022-10-02T16:05:00Z,2022-10-02T16:38:31Z,2022-10-16T16:38:31Z,READY,2022-10-02T16:38:31Z,2022-10-02T16:52:15Z
5,453e-542c-9eadab3b-9c21-75061b66043d,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Jsonl-...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...",[4214-751a-bc915a5a-9b4a-5ac1a3db379d],1,2022-09-25T16:05:00Z,2022-10-02T16:05:00Z,2022-10-02T16:32:59Z,2022-10-16T16:32:59Z,READY,2022-10-02T16:32:59Z,2022-10-02T16:50:27Z
6,48e8-f3ba-0ab37143-99be-f493d12a2616,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Jsonl-...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[41a2-0374-21645fe4-9172-00c126ac56bb, 4f4b-57...",2,1970-01-01T00:00:00Z,2022-10-02T16:05:00Z,2022-10-02T16:35:30Z,2022-10-16T16:35:30Z,READY,2022-10-02T16:35:30Z,2022-10-02T16:52:15Z
7,4afd-f645-d7d3effe-bb5d-2eeb6058ea5d,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Jso...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[40c8-0ead-7994e160-b74f-2ee615fa3426, 45bc-7d...",2,1970-01-01T00:00:00Z,2022-10-02T16:05:00Z,2022-10-02T16:19:28Z,2022-10-16T16:19:28Z,READY,2022-10-02T16:19:28Z,2022-10-02T16:52:15Z
8,4cd3-94c9-0965c678-ba0f-2d7026f45718,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Csv...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[41e0-c68e-c0a5d351-8808-06732281013e, 436a-39...",5,2022-09-25T16:05:00Z,2022-10-02T16:05:00Z,2022-10-02T16:22:30Z,2022-10-16T16:22:29Z,READY,2022-10-02T16:22:30Z,2022-10-02T16:50:27Z
9,4da1-bf90-bfb4c7eb-b69d-5c65701555e7,Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Jsonl-...,bulk-ESG,4288-ebb6-93372235-acb2-89882a826af1,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[4592-033f-7e42e86e-9ec6-bd956cf70688, 4d47-2e...",2,2022-09-25T16:05:00Z,2022-10-02T16:05:00Z,2022-10-02T16:43:58Z,2022-10-16T16:43:57Z,READY,2022-10-02T16:43:58Z,2022-10-02T16:50:27Z


### Iterate over Latest ESG FileSets and Request the Latest ESG Files (Init and Delta)

In [33]:
print("List of FileSet Ids to be streamed by this step:")
for item in ESG_FILESET_RESP['value']:
    print ('\t'+item['id'])
    # Request File Details for the FileSets of interest
    jsonFullRespFile = requestFileDetails(accessToken, item['id']);
    print('\t\tList of Files:')
    for item2 in jsonFullRespFile['value']:
        print ('File name: ' +item2['filename'])
    # Request download per file Id and into fileName
    print('Starting download ... ')
    for item2 in jsonFullRespFile['value']:
        print ('Streaming File: ' +item2['filename'])
        requestFileDownload(accessToken, item2['id'],item2['filename']);

List of FileSet Ids to be streamed by this step:
	4081-6dd8-09c497f8-bdf5-66a9b303b0c4
Obtaining File details for FileSet= 4081-6dd8-09c497f8-bdf5-66a9b303b0c4 ...
(If result is Response=400, make sure that fileSetId is set with a valid value...)
Reading the token from: token.txt
Raw response=
<Response [200]>
		List of Files:
File name: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16:11:18.777Z-part4.csv.gz
File name: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16:11:18.777Z-part0.csv.gz
File name: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16:11:18.777Z-part1.csv.gz
File name: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16:11:18.777Z-part3.csv.gz
File name: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16:11:18.777Z-part5.csv.gz
File name: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16:11:18.777Z-part6.csv.gz
File name: Bulk-ESG-Global-Raw-Full-SchemeB-v1-Social-Init-2022-10-02T16:11:18.777Z-part2

### Get File Location (Step 1 of 2)

In [41]:
import shutil

FILES_STREAM_ENDPOINT_START = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files/'
DIRECT_URL = ''
 
def requestFileLocation(token, fileId):   
    
    FILES_STREAM_ENDPOINT = FILES_STREAM_ENDPOINT_START + fileId+ '/stream?doNotRedirect=true'    
    print("Obtaining File ... " + FILES_STREAM_ENDPOINT)
  
    filename = FILE_NAME
    chunk_size = 1000
    
    headers = {
            'Authorization': 'Bearer ' + token,
            'cache-control': "no-cache",
            'Accept': '*/*'
    }
        
    response = requests.request("GET", FILES_STREAM_ENDPOINT, headers=headers, stream=False, allow_redirects=False)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET",FILES_STREAM_ENDPOINT, headers=headers, stream=False, allow_redirects=False)

         
    print('Response code=' + str(response.status_code));
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        print('Parsed json response=');
        print(json.dumps(jsonFullResp, indent=2));
        DIRECT_URL = jsonFullResp['url'];
        print('File Direct URL is: '  +str(DIRECT_URL)+ '|||');
        
        return jsonFullResp['url'];
    else:
        return 'Error response: '+ response.text


DIRECT_URL = requestFileLocation(accessToken, FILE_ID);

Obtaining File ... https://api.refinitiv.com/file-store/v1/files/42fe-6659-b43c3f44-887a-112dfe7c298a/stream?doNotRedirect=true
Reading the token from: token.txt
Response code=403


### Download File From File Location (Step 2 of 2)

In [48]:
from urllib.parse import urlparse, parse_qs
def requestDirectFileDownload(token, directUrl, fileName):   
    
    global DIRECT_URL
    print("Obtaining File from URL... " + directUrl)
    
    #Parse out URL parameters for submission into requests
    url_obj = urlparse(DIRECT_URL)
    parsed_params = parse_qs(url_obj.query)
    # extract the URL without query parameters
    parsed_url = url_obj._replace(query=None).geturl()

    response = requests.get(parsed_url, params=parsed_params,stream=True)
        
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.get(parsed_url, params=query)

         
    print('Response code=' + str(response.status_code));        
  
    filename = 'another_'+fileName    
    
    if response.status_code == 200:
        print('Processing...')
        with open(filename, 'wb') as fd:
            shutil.copyfileobj(response.raw, fd) 

        print('Look for gzipped file named: '+ filename + ' in current directory')
        response.connection.close()
        
    return; 


requestDirectFileDownload(accessToken, DIRECT_URL, FILE_NAME);

Obtaining File from URL... https://a206464-prod-esg.s3.amazonaws.com/ESG_Sources/2021/01/10/RFT-ESG-Sources-Current-Delta-2021-01-10.jsonl.gz?x-request-Id=eb92c291-b7ed-45f5-b9b1-7426c1eeb5de&x-package-id=4867-9a46-216e838a-9241-8fc3561b51ef&x-client-app-id=GE-A-01103867-3-603&x-file-name=RFT-ESG-Sources-Current-Delta-2021-01-10.jsonl.gz&x-fileset-id=4022-f0c0-e3968404-9146-1b496d4e11b4&x-bucket-name=ESG&x-uuid=GENTC-25929&x-file-Id=4f94-ff55-90b727d0-b896-3ecd59dd8855&x-fileset-name=RFT-ESG-Sources-Current-Delta-2021-01-10&x-event-external-name=cfs-claimCheck-download&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEB8aCXVzLWVhc3QtMSJHMEUCIB%2FU%2BDwNMBdiyBPP3VRMHSwOIYPZvjx9ER0ngyT2aRIwAiEAutDJejT0pDIo5mcs28WLDxhtlIa1zCEbalRshMfPkd0qyQMIFxACGgw2NDIxNTcxODEzMjYiDKSUphGcBSnz%2Ff1GkiqmA6wj5QKW2GDND4oMFe3cNYae8Ka1LliYadEM1Dcy9kuxemBBkQ6AZmki8OUFlSoz0hTKbLtnYQYqRVSuAVGqZLOCIpzw4qQo2QLnd6f7F24nsZxjKpH2B1Vlk8dQr5nWvdWDHN4xNvaH0ZQKj%2BdW58rNC4Hr88Q8jn01FLqIyLbo510S32u45kMtglYpTwia9%2FSnMqxHhapl4FI4Pkdq5